## 1) ¿Cuáles son los PEAS de este problema? (Performance, Environment, Actuators, Sensors)
### Performance
Discos ordenados de mayor a menor, reduciendo la cantidad de movimientos (tiempo mínimo)

### Environment
Varillas, discos, reglas de movimientos.  

### Actuators
Mover el disco de una varilla a otro cumpliendo las siguientes reglas:  
1. Solo se puede mover un disco a la vez.  
2. Cada movimiento consiste en tomar el disco superior de una de las pilas y colocarlo sobre otra pila o sobre una varilla vacía.  
3. Ningún disco puede colocarse sobre uno que sea más pequeño.  

### Sensors
Posicion de los discos en las varillas.

## 2) ¿Cuáles son las propiedades del entorno de trabajo?

Tomando las propiedades vistas en clase podemos decir que para este caso aplican las siguientes:

- Totalmente observable
- Determinista
- Secuencial ?
- Estático
- Discreto
- Agente individual

##  3)  En el contexto de este problema, defina los siguientes conceptos:
### Estado
El estado se define como la terna ordenada $(r_1, r_2, r_3)$ donde $r_i$ es una lista representando el contenido de la varilla $i$. Cada disco se representa con un número del 1 al 5 aludiendo a su tamaño, siendo el 1 el más pequeño y el 5 el más grande.

Nótese que para que un estado sea válido además deberá cumplir:

1. Si $r_i = [d_1, ..., d_k] \Rightarrow  d_1 > ... > d_k$ (Las varillas se encuentran ordenadas ascendentemente)
2. $\cap_{i=1}^3 r_i = \phi$ (No existen discos repetidos entre varillas)
3. $\cup_{i=1}^3 r_i = \{1, 2, 3, 4, 5\}$ (No existen discos faltantes)

### Espacio de Estados
Es el conjunto $S$ de todos los estados válidos, es decir, todos los estados que cumplen 1., 2. y 3. En este caso la cardinalidad del espacio será de $|S| = 3^5 = 243$ estados posibles

### Árbol de Búsqueda
Árbol (estructura de datos) enraizado en el estado inicial del agente $E_{initial} = ([5, 4, 3, 2, 1], [], [])$. Cada nodo no inicial implica los caminos que recorrió el agente (los movimientos realizados) y que, pueden, arribar a un nodo solución. La forma de expansión del mismo depende del algoritmo de búsqueda utilizado, en este caso, DFS.

### Nodo de Búsqueda
Un nodo de búsqueda es representado por cuatro componentes:

- Un estado del espacio de estados que corresponde al nodo.
- Un nodo diferente del árbol de búsqueda del que se ha generado este nodo.
- La acción que se aplicará al padre para generar el nodo. (Qué disco se moverá del estado del nodo padre para arribar a este nodo)
- El costo de un camino desde el nodo inicial al nodo. (Cantidad de discos movidos para arribar a este nodo desde el inicial)

### Objetivo
Estado: $E_{goal} = ([], [] ,[5, 4, 3, 2, 1])$.

### Acción
La acción se define como la tupla ordenada $(r_{in}, r_{out})$ donde $r_{in}, r_{out} \in \{1,2,3\}$ representan mover el disco de la varilla $r_{in}$ a la varilla $r_{out}$.

Nótese que para que una acción sea válida además deberá cumplir:
- $r_{r_{in}} \neq []$ (No se permite mover un disco inexistente)
- $top(r_{r_{in}}) < top(r_{r_{out}})$, donde $top(r_i)$ denota el último elemento de la lista $r_i$ (Consistencia entre movimiento y tamaño)

### Frontera 
La frontera separa dos regiones del grafo de búsqueda, aquella que ya fue explorada por el algoritmo y aquella que no. Puede entenderse como un conjunto de nodos aún no explorados.

In [25]:
from aima_libs.hanoi_states import StatesHanoi, ActionHanoi, ProblemHanoi
from aima_libs.tree_hanoi import NodeHanoi

In [26]:
def depth_first_search(number_disks=5):
    # Inicializamos el problema
    list_disks = [i for i in range(5, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    frontier = [NodeHanoi(problem.initial)]  #Cola LIFO con el nodo inicial
    explored = set()  #Conjunto de estados ya visitados

    node_explored = 0
    
    while len(frontier) != 0:
        node = frontier.pop()
        node_explored += 1
        
        explored.add(node.state)  # Verificamos si llegamos al objetivo
        
        if problem.goal_test(node.state):
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        # Agregamos a la frontera los nodos sucesores que no hayan sido visitados
        for next_node in node.expand(problem):
            if next_node.state not in explored and next_node not in frontier:
                frontier.append(next_node)

    # Si no se encuentra solución, devolvemos métricas igualmente
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth, # OBS: Si no se encontró la solución, este valor solo tiene sentido en breadth_first_search, en otros casos se debe ir llevando registro de cual fue la máxima profundidad
        "cost_total": None,
    }
    return None, metrics

In [27]:
solution, metrics = depth_first_search(number_disks=5)

In [28]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 82
states_visited: 82
nodes_in_frontier: 41
max_depth: 81
cost_total: 81.0


In [29]:
for nodos in solution.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 | 2 | 3 1>
<Node HanoiState: 5 4 2 |  | 3 1>
<Node HanoiState: 5 4 2 | 1 | 3>
<Node HanoiState: 5 4 | 1 | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 | 4 | 3 2 1>
<Node HanoiState: 5 | 4 1 | 3 2>
<Node HanoiState: 5 2 | 4 1 | 3>
<Node HanoiState: 5 2 | 4 | 3 1>
<Node HanoiState: 5 | 4 2 | 3 1>
<Node HanoiState: 5 | 4 2 1 | 3>
<Node HanoiState: 5 3 | 4 2 1 | >
<Node HanoiState: 5 3 | 4 2 | 1>
<Node HanoiState: 5 3 2 | 4 | 1>
<Node HanoiState: 5 3 2 | 4 1 | >
<Node HanoiState: 5 3 | 4 1 | 2>
<Node HanoiState: 5 3 | 4 | 2 1>
<Node HanoiState: 5 | 4 3 | 2 1>
<Node HanoiState: 5 | 4 3 1 | 2>
<Node HanoiState: 5 2 | 4 3 1 | >
<Node HanoiState: 5 2 | 4 3 | 1>
<Node HanoiState: 5 | 4 3 2 | 1>
<Node HanoiState: 5 | 4 3 2 1 | >
<Node HanoiState:  | 4 3 2 1 | 5>
<Node HanoiState:  | 4 3 2 | 5 1

In [30]:
for act in solution.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from

## 5) ¿Cuál es la complejidad teórica en tiempo y memoria del algoritmo elegido?
Complejidad Tiempo -> O(b^d) 
Complejidad Memoria -> O(b . d)

Siendo b -> branches en que se va abriendo. En este caso abre de a 2. 

Siendo d -> profundidad maxima del arbol.

## 6) A nivel de implementación, ¿cuánto tiempo y memoria utiliza el algoritmo? (Se recomienda ejecutarlo 10 veces y calcular el promedio y el desvío estándar de ambas métricas).

In [31]:
import timeit
import tracemalloc
import statistics

times = []
memories = []

for _ in range(10):
    
    tracemalloc.start() 
    
    time = timeit.timeit(lambda: depth_first_search(number_disks=5), number=1)
    
    # Añado al array tiempos el tiempo de cada iteracion
    times.append(time)
    
    current, peak = tracemalloc.get_traced_memory() 
    tracemalloc.stop() 
    
    # Añado al array memoria el pico de memoria de cada iteracion
    memories.append(peak / (1024*1024))  # Convertir a MB
    
    # print(f"Iteracion {_ + 1}: Memoria pico: {round(peak / (1024*1024),2)} MB, Tiempo: {time:.4f} segundos")
    
print(f"Tiempo promedio: {statistics.mean(times):.4f} segundos")
print(f"Desvío estándar del tiempo: {statistics.stdev(times):.4f} segundos")
print(f"Memoria promedio: {statistics.mean(memories):.2f} MB")
print(f"Desvío estándar de la memoria: {statistics.stdev(memories):.4f} MB")    

Tiempo promedio: 0.0158 segundos
Desvío estándar del tiempo: 0.0007 segundos
Memoria promedio: 0.14 MB
Desvío estándar de la memoria: 0.0039 MB


## 7) Si la solución óptima es de $2^k - 1$ movimientos (siendo k el número de discos), ¿qué tan lejos está la solución encontrada por el algoritmo implementado de esa solución óptima? (Se recomienda ejecutar al menos 10 veces y usar el promedio de los trayectos obtenidos).

In [32]:
import statistics

max_depths = []
number_disks = 5

for _ in range(5):
    
    solution, metrics = depth_first_search(number_disks)
    #print(f"Solución encontrada: {metrics['max_depth']}")
    max_depths.append(metrics['max_depth'])
    
print(f"Profundidad máxima promedio: {statistics.mean(max_depths):.2f}")
optimal_depth = 2 ** number_disks  - 1
print(f"Profundidad óptima: {optimal_depth}")
print(f"Diferencia con la óptima: {abs(statistics.mean(max_depths) - optimal_depth):.2f}")

Profundidad máxima promedio: 81.00
Profundidad óptima: 31
Diferencia con la óptima: 50.00
